In [1]:
import os

In [2]:
from glob import glob

In [3]:
import matplotlib.pyplot as plt

In [4]:
import rasterio as rio
from rasterio.plot import plotting_extent

In [5]:
import geopandas as gpd

In [6]:
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

In [7]:
import pyproj

In [23]:
import sys
sys.path.append("C:\\OSGeo4W64\\apps\\qgis\\python\\")
sys.path.append("C:\\OSGeo4W64\\apps\\qgis\\python\\qgis")
sys.path.append("C:\\OSGeo4W64\\apps\\qgis\\")
sys.path.append("C:\\OSGeo4W64\\apps\\Python37\\")
sys.path.append("C:\\OSGeo4W64\\apps\\qgis\\bin\\")
import qgis.utils
from qgis.core import *
print("helo") # console output: helo
QgsApplication.setPrefixPath("/Applications/QGIS.app/", True)
QgsApplication.initQgis()
print(QgsApplication) # console output: <class 'qgis.core.QgsApplication'>
print(qgis.utils.iface) # = console output: none
aLayer = qgis.utils.iface.activeLayer()
print(aLayer.name())
QgsApplication.exitQgis()

ImportError: DLL load failed: The specified procedure could not be found.

In [21]:
print(sys.path)

['C:\\Users\\Hello\\Documents\\remote-sensing\\APIs\\SentinelSat', 'C:\\ProgramData\\Anaconda3\\python37.zip', 'C:\\ProgramData\\Anaconda3\\DLLs', 'C:\\ProgramData\\Anaconda3\\lib', 'C:\\ProgramData\\Anaconda3', '', 'C:\\Users\\Hello\\AppData\\Roaming\\Python\\Python37\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Hello\\.ipython', '/Applications/QGIS.app/Contents/Resources/python', 'C:\\OSGeo4W64\x07pps\\qgis\\python\\qgis\\core', 'C:\\OSGeo4W64\x07pps\\qgis\\python', 'C:\\OSGeo4W64\\apps\\qgis\\python\\', 'C:\\OSGeo4W64\\apps\\qgis\\python\\', 'C:\\OSGeo4W64\\apps\\qgis\\python\\', 'C:\\OSGeo4W64\\apps\\qgis\\python\\qgis', 'C:\\OSGeo4W64\\apps\\qgis\\']
